In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical

import numpy as np
import matplotlib.pyplot as plt
import os

print(tf.__version__)
print(keras.__version__)

2.1.0
2.2.4-tf


## 1. Set hyper parameters

In [2]:
lr = 0.001
train_epoch = 15
bsize = 100

tf.random.set_seed(777)

#### + Creating a checkpoint directory

In [3]:
cur_dir = os.getcwd()
ckpt_dir = 'checkpoints'
model_dir = 'mnist_cnn_subclass'

save_dir = os.path.join(cur_dir, ckpt_dir, model_dir)
os.makedirs(save_dir, exist_ok=True)

ckpt_prefix = os.path.join(ckpt_dir, model_dir)

## 2. Make a data pipelining

In [4]:
mnist = keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [5]:
X_train = X_train.astype(np.float32) / 255.
X_train = np.expand_dims(X_train, axis=-1)

y_train = to_categorical(y_train, num_classes=10)

train_dataset = tf.data.Dataset.from_tensor_slices(
                    (X_train, y_train)).\
                    shuffle(buffer_size=100000).batch(bsize)


X_test = X_test.astype(np.float32) / 255.
X_test = np.expand_dims(X_test, axis=-1)

y_test = to_categorical(y_test, num_classes=10)

test_dataset = tf.data.Dataset.from_tensor_slices(
                    (X_test, y_test)).\
                    shuffle(buffer_size=100000).batch(bsize)

## 3. Build a neural network model

In [6]:
class MNISTModel(tf.keras.Model):
    def __init__(self):
        super(MNISTModel, self).__init__()
        self.conv1 = keras.layers.Conv2D(filters=32, kernel_size=3,
                                         strides=1, padding='same',
                                         activation=tf.nn.relu)
        self.pool1 = keras.layers.MaxPool2D(pool_size=2,
                                            padding='same')
        self.conv2 = keras.layers.Conv2D(filters=64, kernel_size=3,
                                         strides=1, padding='same',
                                         activation=tf.nn.relu)
        self.pool2 = keras.layers.MaxPool2D(pool_size=2,
                                            padding='same')
        self.conv3 = keras.layers.Conv2D(filters=128, kernel_size=3,
                                         strides=1, padding='same',
                                         activation=tf.nn.relu)
        self.pool3 = keras.layers.MaxPool2D(pool_size=2,
                                            padding='same')        
        self.pool3_flat = keras.layers.Flatten()
        
        self.dense4 = keras.layers.Dense(units=256,
                                         activation=tf.nn.relu)
        self.drop4 = keras.layers.Dropout(rate=0.4)        
        self.dense5 = keras.layers.Dense(units=10)
        pass
    
    def call(self, inputs, training=False):
        net = self.conv1(inputs)
        net = self.pool1(net)
        net = self.conv2(net)
        net = self.pool2(net)
        net = self.conv3(net)
        net = self.pool3(net)
        net = self.pool3_flat(net)
        net = self.dense4(net)
        net = self.drop4(net)
        net = self.dense5(net)
        return net

In [7]:
model = MNISTModel()
temp_inputs = keras.Input(shape=(28, 28, 1))
model(temp_inputs)
model.summary()

Model: "mnist_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 128)         73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 2048)              

## 4. Define a loss function

In [8]:
@tf.function
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean( 
                tf.keras.losses.categorical_crossentropy(
                    y_true=labels, y_pred=logits, from_logits=True))
    return loss

## 5. Calculate a gradient

In [9]:
@tf.function
def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

## 6. Select an optimizer

In [10]:
optimizer = tf.optimizers.Adam(learning_rate=lr)

## 7. Define a metric for model's performance

In [11]:
@tf.function
def evaluate(model, images, labels):
    logits = model(images, training=False)
    correct_prediction = tf.equal( tf.argmax(logits, axis=1),
                                   tf.argmax(labels, axis=1) )
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, 
                                      dtype=tf.float32))
    return accuracy

## 8. Make a checkpoint for saving

In [12]:
checkpoint = tf.train.Checkpoint(cnn=model)

## 9. Train and Validate a neural network model

In [13]:
@tf.function
def train(model, images, labels):
    grads = grad(model, images, labels)
    optimizer.apply_gradients(
        grads_and_vars=zip(grads, model.trainable_variables))
    pass


for epoch in range(train_epoch):
    avg_loss = 0.; avg_train_acc = 0.; avg_test_acc = 0.
    train_step = 0; test_step = 0
    
    for images, labels in train_dataset:
        train(model, images, labels)
        loss = loss_fn(model, images, labels)
        avg_loss = avg_loss + loss
        acc = evaluate(model, images, labels)
        avg_train_acc = avg_train_acc + acc
        train_step += 1
    avg_loss = avg_loss / train_step
    avg_train_acc = avg_train_acc / train_step
    
    for images, labels in test_dataset:
        acc = evaluate(model, images, labels)
        avg_test_acc = avg_test_acc + acc
        test_step += 1
    avg_test_acc = avg_test_acc / test_step
    
    print(f'Epoch: {epoch+1:3}  |  loss = {avg_loss:10.8f}  |  '+
          f'train_accuracy = {avg_train_acc:6.4f}  |  '+
          f'test_accuracy = {avg_test_acc:6.4f}')
    checkpoint.save(file_prefix=ckpt_prefix)

print()
print('Learning Finished !')

Epoch:   1  |  loss = 0.17375422  |  train_accuracy = 0.9564  |  test_accuracy = 0.9868
Epoch:   2  |  loss = 0.04280969  |  train_accuracy = 0.9891  |  test_accuracy = 0.9897
Epoch:   3  |  loss = 0.02930635  |  train_accuracy = 0.9929  |  test_accuracy = 0.9918
Epoch:   4  |  loss = 0.02045051  |  train_accuracy = 0.9952  |  test_accuracy = 0.9926
Epoch:   5  |  loss = 0.01594689  |  train_accuracy = 0.9968  |  test_accuracy = 0.9914
Epoch:   6  |  loss = 0.01256046  |  train_accuracy = 0.9972  |  test_accuracy = 0.9901
Epoch:   7  |  loss = 0.00999002  |  train_accuracy = 0.9977  |  test_accuracy = 0.9943
Epoch:   8  |  loss = 0.00781305  |  train_accuracy = 0.9982  |  test_accuracy = 0.9920
Epoch:   9  |  loss = 0.00661438  |  train_accuracy = 0.9987  |  test_accuracy = 0.9916
Epoch:  10  |  loss = 0.00514866  |  train_accuracy = 0.9989  |  test_accuracy = 0.9940
Epoch:  11  |  loss = 0.00431266  |  train_accuracy = 0.9992  |  test_accuracy = 0.9934
Epoch:  12  |  loss = 0.00376989